In [43]:
import numpy as np
import pandas as pd

df_learning = pd.read_csv("../data-source/dataset.csv")

In [44]:
import jieba


def cutReview(x): return ' '.join(jieba.lcut(str(x), cut_all=False))


df_learning.loc[:, '类型'] = df_learning['类型'].fillna(
    '')
df_learning.loc[:, '交易对方'] = df_learning['交易对方'].fillna(
    '').apply(cutReview)
df_learning.loc[:, '商品名称'] = df_learning['商品名称'].fillna(
    '').apply(cutReview)


# 模型训练

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from scipy.sparse import hstack

# 去除词频小于2的词
countvec = CountVectorizer(min_df=2)

# 构建特征向量
product_vector = countvec.fit_transform(df_learning['商品名称'])
saler_vector = countvec.fit_transform(df_learning['交易对方'])

# 合并特征向量
feature_vector = hstack((product_vector, saler_vector))

# 特征降维
pca = PCA(n_components=100)
feature_vector_pca = pca.fit_transform(feature_vector.toarray())

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 按照 7:3 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(
    feature_vector_pca, df_learning['类型'], test_size=0.3)

svm_model = SVC(kernel='rbf', probability=True, random_state=1)
svm_model.fit(x_train, y_train)

y_predict = svm_model.predict(x_test)

# 计算准确度
acc = accuracy_score(y_test, y_predict)
print("训练模型准确率为：{:.2f}%".format(acc * 100))


训练模型准确率为：69.76%


# 预测集填入

In [47]:
df_new = pd.read_csv("../data-source/predict.csv")

df_predict = df_new.copy()

def cutReview(x): return ' '.join(jieba.lcut(str(x), cut_all=False))

df_predict.loc[:, '交易对方'] = df_predict['交易对方'].fillna(
    '').apply(cutReview)
df_predict.loc[:, '商品名称'] = df_predict['商品名称'].fillna(
    '').apply(cutReview)

df_predict

,交易时间,类型,金额(元),收/支,交易来源,交易对方,商品名称,备注
0,2022/8/1 11:56:03,NaN,16.56,支出,微信支付,顺丰速运,顺丰速 运散 单 运费,NaN
1,2022/8/2 14:37:11,NaN,1279.72,支出,支付宝,厦门航空 有限公司,厦门航空,NaN
2,2022/8/2 15:42:14,NaN,260.00,支出,支付宝,美团,美团 订单 - 22080211100300000024761828600572,来北京住的第一晚
3,2022/8/2 19:47:47,NaN,15.97,支出,微信支付,株洲市 中心医院,株洲市 中心医院,核酸
4,2022/8/2 19:47:47,NaN,15.97,支出,微信支付,株洲市 中心医院,株洲市 中心医院,NaN
...,...,...,...,...,...,...,...,...
1164,2023/1/29 16:54:37,NaN,28.77,支出,工商银行,,咖啡,NaN
1165,2023/1/29 20:12:11,NaN,11.40,支出,微信支付,靳杰,收款 方 备注 : 二维码 收款,NaN
1166,2023/1/30 16:47:15,NaN,19.90,支出,支付宝,橙子 * * 店,橙子 快 跑 全麦 黑麦 吐司 减低 脂代餐 孕妇 粗粮 早餐 无添 蔗糖 零食 面包,NaN
1167,2023/1/30 17:47:39,NaN,2.90,支出,支付宝,天津市 新 发地 菜篮子 店,商户 张国欣,NaN


In [48]:
# 去除词频小于2的词
countvec = CountVectorizer(min_df=2)

# 构建特征向量
product_vector = countvec.fit_transform(df_predict['商品名称'])
saler_vector = countvec.fit_transform(df_predict['交易对方'])

# 合并特征向量
feature_vector = hstack((product_vector, saler_vector))

# 特征降维
pca = PCA(n_components=100)
feature_vector_pca = pca.fit_transform(feature_vector.toarray())

# 结果预测

In [49]:
confidence_threshold = 0.35

y_predict = svm_model.predict(feature_vector_pca)
confidence_scores = svm_model.predict_proba(feature_vector_pca)

# 根据阈值过滤不可信的结果
for i in range(len(y_predict)):
    if np.max(confidence_scores[i]) < confidence_threshold:
        y_predict[i] = ''

df_new['类型'] = y_predict

df_new


,交易时间,类型,金额(元),收/支,交易来源,交易对方,商品名称,备注
0,2022/8/1 11:56:03,人情,16.56,支出,微信支付,顺丰速运,顺丰速运散单运费,NaN
1,2022/8/2 14:37:11,,1279.72,支出,支付宝,厦门航空有限公司,厦门航空,NaN
2,2022/8/2 15:42:14,餐饮,260.00,支出,支付宝,美团,美团订单-22080211100300000024761828600572,来北京住的第一晚
3,2022/8/2 19:47:47,人情,15.97,支出,微信支付,株洲市中心医院,株洲市中心医院,核酸
4,2022/8/2 19:47:47,人情,15.97,支出,微信支付,株洲市中心医院,株洲市中心医院,NaN
...,...,...,...,...,...,...,...,...
1164,2023/1/29 16:54:37,人情,28.77,支出,工商银行,NaN,咖啡,NaN
1165,2023/1/29 20:12:11,餐饮,11.40,支出,微信支付,靳杰,收款方备注:二维码收款,NaN
1166,2023/1/30 16:47:15,生活,19.90,支出,支付宝,橙子**店,橙子快跑全麦黑麦吐司减低脂代餐孕妇粗粮早餐无添蔗糖零食面包,NaN
1167,2023/1/30 17:47:39,餐饮,2.90,支出,支付宝,天津市新发地菜篮子店,商户张国欣,NaN


In [50]:
df_new.to_csv("../out/predict_result.csv", index=False, header=True, encoding='utf-8')